In [27]:
from datetime import datetime
import traceback
import functools
import multiprocessing
from multiprocessing import Pool

import pytz
import time
import os
import sys
from pathlib import Path

import numpy as np
import scipy
import pandas as pd

import matplotlib
matplotlib.use('nbagg')
from matplotlib import style
style.use('ggplot')
import matplotlib.pyplot as plt

%load_ext autoreload
%autoreload 2

from astropy import stats
from astropy.io import fits, ascii
from astropy.table import Column

import astropy.units as u
from astropy.io import fits
from mmtwfs.wfs import WFSFactory

tz = pytz.timezone("America/Phoenix")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
def get_traceback(f):
    @functools.wraps(f)
    def wrapper(*args, **kwargs):
        try:
            return f(*args, **kwargs)
        except Exception as ex:
            ret = '#' * 60
            ret += "\nException caught:"
            ret += "\n"+'-'*60
            ret += "\n" + traceback.format_exc()
            ret += "\n" + '-' * 60
            ret += "\n"+ "#" * 60
            print(sys.stderr, ret)
            sys.stderr.flush()
            raise ex
 
    return wrapper

In [3]:
# instantiate all of the WFS systems...
wfs_keys = ['f9', 'newf9', 'f5', 'mmirs', 'binospec']
wfs_systems = {}
wfs_names = {}
for w in wfs_keys:
    wfs_systems[w] = WFSFactory(wfs=w)
    wfs_names[w] = wfs_systems[w].name
plt.close('all')

# give mmirs a default
wfs_systems['mmirs'].default_mode = 'mmirs1'

# map f9 to oldf9
wfs_systems['oldf9'] = wfs_systems['f9']

In [4]:
def check_image(f, wfskey=None):
    hdr = {}
    with fits.open(f) as hdulist:
        for h in hdulist:
            hdr.update(h.header)
        data = hdulist[-1].data
    
    # if wfskey is None, figure out which WFS from the header info...
    if wfskey is None:
        # check for MMIRS
        if 'WFSNAME' in hdr:
            if 'mmirs' in hdr['WFSNAME']:
                wfskey = 'mmirs'
        if 'mmirs' in f.name:
            wfskey = 'mmirs'

        # check for binospec
        if 'bino' in f.name:
            wfskey = 'binospec'
        if 'ORIGIN' in hdr:
            if 'Binospec' in hdr['ORIGIN']:
                wfskey = 'binospec'

        # check for new F/9
        if 'f9wfs' in f.name:
            wfskey = 'newf9'
        if 'OBSERVER' in hdr:
            if 'F/9 WFS' in hdr['OBSERVER']:
                wfskey = 'newf9'
        if wfskey is None and 'CAMERA' in hdr:
            if 'F/9 WFS' in hdr['CAMERA']:
                wfskey = 'newf9'

        # check for old F/9
        if 'INSTRUME' in hdr:
            if 'Apogee' in hdr['INSTRUME']:
                wfskey = 'oldf9'
        if 'DETECTOR' in hdr:
            if 'Apogee' in hdr['DETECTOR']:
                wfskey = 'oldf9'

        # check for F/5 (hecto)
        if wfskey is None and 'SEC' in hdr:  # mmirs has SEC in header as well and is caught above
            if 'F5' in hdr['SEC']:
                wfskey = 'f5'
        if Path(f.parent / "F5").exists():
            wfskey = 'f5'
            
    if wfskey is None:
        # if wfskey is still None at this point, whinge.
        print(f"Can't determine WFS for {f.name}...")

    if 'AIRMASS' not in hdr:
        if 'SECZ' in hdr:
            hdr['AIRMASS'] = hdr['SECZ']
        else:
            hdr['AIRMASS'] = np.nan

    if 'EXPTIME' not in hdr:
        hdr['EXPTIME'] = np.nan
            
    # we need to fix the headers in all cases to have a proper DATE-OBS entry with
    # properly formatted FITS timestamp.  in the meantime, this hack gets us what we need 
    # for analysis in pandas.
    dtime = None
    if 'DATEOBS' in hdr:
        dateobs = hdr['DATEOBS']
        if 'UT' in hdr:
            ut = hdr['UT'].strip()
        elif 'TIME-OBS' in hdr:
            ut = hdr['TIME-OBS']
        else:
            ut = "07:00:00"  # midnight
        timestring = dateobs + " " + ut + " UTC"
        if '-' in timestring:
            dtime = datetime.strptime(timestring, "%Y-%m-%d %H:%M:%S %Z")
        else:
            dtime = datetime.strptime(timestring, "%a %b %d %Y %H:%M:%S %Z")

    else:
        if wfskey == "oldf9":
            d = hdr['DATE-OBS']
            if '/' in d:
                day, month, year = d.split('/')
                year = str(int(year) + 1900)
                timestring = year + "-" + month + "-" + day + " " + hdr['TIME-OBS'] + " UTC"
            else:
                timestring = d + " " + hdr['TIME-OBS'] + " UTC"
            dtime = datetime.strptime(timestring, "%Y-%m-%d %H:%M:%S %Z")
        else:
            if 'DATE-OBS' in hdr:
                timestring = hdr['DATE-OBS'] + " UTC"
                try:
                    dtime = datetime.strptime(timestring, "%Y-%m-%dT%H:%M:%S.%f %Z")
                except:
                    dtime = datetime.strptime(timestring, "%Y-%m-%dT%H:%M:%S %Z")
                # mmirs uses local time in this header pre-2019
                if wfskey == 'mmirs' and dtime < datetime.fromisoformat("2019-01-01T12:00:00"):
                    local_dt = tz.localize(dtime)
                    dtime = local_dt.astimezone(pytz.utc)
            else:
                dt = datetime.fromtimestamp(f.stat().st_ctime)
                local_dt = tz.localize(dt)
                dtime = local_dt.astimezone(pytz.utc)

    if dtime is None:
        print(f"No valid timestamp in header for {f.name}...")
        obstime = None
    else:
        obstime = dtime.isoformat().replace('+00:00', '')
        
    hdr['WFSKEY'] = wfskey
    hdr['OBS-TIME'] = obstime
    return data, hdr

In [52]:
@get_traceback
def process_image(f):
    """
    Process FITS file, f, to get info we want from the header and then analyse it with the 
    appropriate WFS instance. Return results in a comma-separated line that will be collected 
    and saved in a CSV file.
    """            
    data, hdr = check_image(f)
    
    wfskey = hdr['WFSKEY']
    obstime = hdr['OBS-TIME']
    airmass = hdr['AIRMASS']
    exptime = hdr['EXPTIME']
    az = hdr.get('AZ', np.nan)
    el = hdr.get('EL', np.nan)
    tiltx = hdr.get('TILTX', np.nan)
    tilty = hdr.get('TILTY', np.nan)
    transx = hdr.get('TRANSX', np.nan)
    transy = hdr.get('TRANSY', np.nan)
    focus = hdr.get('FOCUS', np.nan)
    if np.isnan(focus) and 'TRANSZ' in hdr:
        focus = hdr.get('TRANSZ', np.nan)
    osst = hdr.get('OSSTEMP', np.nan)
    if 'OUT_T' in hdr:
        outt = hdr.get('OUT_T', np.nan)
    else:
        outt = hdr.get('T_OUT', np.nan)
    if 'CHAM_T' in hdr:
        chamt = hdr.get('CHAM_T', np.nan)
    else:
        chamt = hdr.get('T_CHAM', np.nan)
    
    # being conservative here and only using data that has proper slope determination
    # and wavefront solution. also want to get statistics on the quality of the wavefront fits.
    try:
        results = wfs_systems[wfskey].measure_slopes(str(f), plot=False)
    except:
        print(f"Problem analyzing {f.name}...")
        results = {}
        results['slopes'] = None

    if results['slopes'] is not None:
        zresults = wfs_systems[wfskey].fit_wavefront(results, plot=False)
        zv = zresults['zernike']
        focerr = wfs_systems[wfskey].calculate_focus(zv)
        cc_x_err, cc_y_err = wfs_systems[wfskey].calculate_cc(zv)
        line = f"{obstime},{wfskey},{f.name},{exptime},{airmass},{az},{el},{osst},{outt},{chamt},{tiltx},{tilty},{transx},{transy},{focus},{focerr.value},{cc_x_err.value},{cc_y_err.value},{results['xcen']},{results['ycen']},{results['seeing'].value},{results['raw_seeing'].value},{results['fwhm']},{zresults['zernike_rms'].value},{zresults['residual_rms'].value}\n"
        zfile = f.parent / (f.stem + ".reanalyze.zernike")
        zresults['zernike'].save(filename=zfile)
        spotfile = f.parent / (f.stem + ".spots.csv")
        results['spots'].write(spotfile, overwrite=True)
        return line
    else:
        return None

#rootdir = Path("/Users/tim/MMT/wfsdat")
rootdir = Path("/Volumes/LaCie 8TB/wfsdat")
#rootdir = Path("/Volumes/Seagate2TB/wfsdat")
#rootdir = Path("/mnt/d/wfsdat")

In [54]:
dirs = sorted(list(rootdir.glob("2019*")))  # pathlib, where have you been all my life!
csv_header = "time,wfs,file,exptime,airmass,az,el,osst,outt,chamt,tiltx,tilty,transx,transy,focus,focerr,cc_x_err,cc_y_err,xcen,ycen,seeing,raw_seeing,fwhm,wavefront_rms,residual_rms\n"
for d in dirs:
    if d.is_dir():
        if Path.exists(d / "reanalyze_results.csv"):
            print("Already processed %s..." % d.name)
        else:
            try:
                lines = []
                lines.append(csv_header)
                night = int(d.name)  # valid WFS directories are ints of the form YYYYMMDD. if not this form, int barfs
                msg = "checking %d... " % night
                fitsfiles = d.glob("*.fits")
                print(msg)
#                 for f in fitsfiles:
#                     print("Processing %s..." % f)
#                     process_image(f)
                nproc = 6 # my mac mini's i7 has 6 cores and py37 can use hyperthreading for more... 
                with Pool(processes=nproc) as pool:  # my mac mini's i7 has 6 cores... 
                    plines = pool.map(process_image, fitsfiles)  # plines comes out in same order as fitslines!
                plines = list(filter(None.__ne__, plines))  # trim out any None entries
                lines.extend(plines)
                with open(d / "reanalyze_results.csv", "w") as f:
                    f.writelines(lines)
            except ValueError as e:  # this means running int(d.name) failed so it's not a valid directory...
                print(f"Skipping %s... ({e})" % d.name)

Already processed 20190101...
Already processed 20190102...
Already processed 20190103...
Already processed 20190104...
Already processed 20190105...
Already processed 20190106...
Already processed 20190107...
Already processed 20190108...
Already processed 20190109...
Already processed 20190110...
Already processed 20190111...
Already processed 20190112...
Already processed 20190113...
Already processed 20190114...
Already processed 20190115...
Already processed 20190116...
Already processed 20190117...
Already processed 20190118...
Already processed 20190119...
Already processed 20190120...
Already processed 20190121...
Already processed 20190122...
Already processed 20190123...
Already processed 20190124...
Already processed 20190125...
Already processed 20190126...
Already processed 20190127...
Already processed 20190128...
Already processed 20190129...
Already processed 20190130...
Already processed 20190131...
Already processed 20190201...
Already processed 20190202...
Already pr

In [ ]:
process_image(Path("/Volumes/LaCie 8TB/wfsdat/20190423/manual_wfs_0005.fits"))

In [34]:
f = Path("/Volumes/Seagate2TB/wfsdat/20190107/reanalyze_results.csv")
t = ascii.read(f)
az = Column(np.zeros(len(t)), name='az')
el = Column(np.zeros(len(t)), name='el')
osst = Column(np.zeros(len(t)), name='osst')
outt = Column(np.zeros(len(t)), name='outt')
chamt = Column(np.zeros(len(t)), name='chamt')
t.add_column(az)
t.add_column(el)
t.add_column(osst)
t.add_column(outt)
t.add_column(chamt)
for r in t:
    with fits.open(f.parent / r['file']) as hl:
        hdr = hl[-1].header
        if np.isnan(r['focus']):
            focus = hdr.get('TRANSZ', np.nan)
            r['focus'] = focus
        a = hdr.get('AZ', np.nan)
        if a < 0:
            a += 360.
        r['az'] = a
        r['el'] = hdr.get('EL', np.nan)
        r['osst'] = hdr.get('OSSTEMP', np.nan)
        r['outt'] = hdr.get('OUT_T', np.nan)
        r['chamt'] = hdr.get('CHAM_T', np.nan)
t

time,wfs,file,exptime,airmass,tiltx,tilty,transx,transy,focus,focerr,cc_x_err,cc_y_err,xcen,ycen,seeing,raw_seeing,fwhm,wavefront_rms,residual_rms,az,el,osst,outt,chamt
str26,str5,str26,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
2019-01-07T08:42:16.357000,newf9,f9wfs_20190107-014234.fits,10.0,1.0103,262.15,67.62,760.62,1263.54,826.94,174.6,33.823,69.341,1754.302456,1459.550976,3.061134,3.080013,23.54969,8306.474561,0.013232,32.06631225,81.82754318,4.88,-0.2,0.135
2019-01-07T08:45:56.894000,newf9,f9wfs_20190107-014608.fits,10.0,1.0093,261.77,66.65,767.54,1256.75,849.24,214.36,-0.0,-3.288,444.093795,374.430704,1.151006,1.157416,10.658541,4436.893868,2776.675973,27.82345466,82.2030739,4.45,-0.1,0.207
2019-01-07T08:47:37.230000,newf9,f9wfs_20190107-014809.fits,30.0,1.0089,202.91,64.33,781.76,909.27,1070.32,-21.56,-0.0,-0.645,381.564102,448.565931,1.165279,1.171491,10.760471,847.297267,2468.092629,25.24997173,82.39181557,4.27,0.0,0.195
2019-01-07T08:49:01.225000,newf9,f9wfs_20190107-014907.fits,5.0,1.0075,202.39,62.98,791.49,899.61,1079.77,-18.28,-2.011,-7.612,407.289992,429.536866,1.089587,1.094483,10.20092,1091.069857,2538.051729,28.94242932,82.98480906,4.08,0.0,0.233
2019-01-07T08:50:34.773000,newf9,f9wfs_20190107-015042.fits,5.0,1.0334,209.35,81.0,665.24,1023.47,1082.29,-28.31,0.0,0.0,356.397331,463.74719,1.095665,1.117478,10.368482,926.229457,2558.610657,12.46391385,75.39840635,3.93,0.4,0.448
2019-01-07T08:50:51.949000,newf9,f9wfs_20190107-015123.fits,30.0,1.0332,209.71,81.75,659.74,1029.2,1080.5,-23.89,-0.0,0.0,364.131662,456.680347,1.175884,1.199154,10.960383,846.156809,2534.172338,11.99886174,75.42877968,3.85,0.6,0.613
2019-01-07T08:52:20.109000,newf9,f9wfs_20190107-015252.fits,30.0,1.0329,209.61,81.65,660.87,1027.74,1069.21,0.0,-0.0,0.0,369.464652,449.093992,1.192806,1.216199,11.083281,692.108567,2472.071811,11.00509259,75.48959072,3.7,0.8,0.762
2019-01-07T08:53:46.527000,newf9,f9wfs_20190107-015418.fits,30.0,1.0327,209.55,81.45,661.93,1026.7,1117.59,-6.04,0.0,0.0,378.922652,444.197907,1.144793,1.167109,10.728755,487.315068,2368.813984,10.02608633,75.54372843,3.53,0.8,0.757
2019-01-07T09:14:58.950000,newf9,f9wfs_20190107-021531.fits,30.0,1.0118,203.92,67.14,761.78,928.25,930.35,236.6,2.224,2.762,400.061331,436.199283,0.91114,0.917575,8.897915,4902.102237,1610.297323,341.01308273,81.24046129,2.4,1.4,1.197


In [41]:
for d in dirs:
    if d.is_dir():
        if Path.exists(d / "reanalyze_results.csv"):
            f = d / "reanalyze_results.csv"
            print(f"fixing {f}...")
            t = ascii.read(f)
            #osst = Column(np.zeros(len(t)), name='osst')
            #outt = Column(np.zeros(len(t)), name='outt')
            #chamt = Column(np.zeros(len(t)), name='chamt')
            #t.add_column(osst)
            #t.add_column(outt)
            #t.add_column(chamt)
            for r in t:
                with fits.open(f.parent / r['file']) as hl:
                    hdr = hl[0].header
                    if np.isnan(r['focus']):
                        focus = hdr.get('TRANSZ', np.nan)
                        r['focus'] = focus
                    a = hdr.get('AZ', np.nan)
                    if a < 0:
                        a += 360.
                    r['az'] = a
                    r['el'] = hdr.get('EL', np.nan)
                    r['osst'] = hdr.get('OSSTEMP', np.nan)
                    if 'OUT_T' in hdr:
                        r['outt'] = hdr.get('OUT_T', np.nan)
                    else:
                        r['outt'] = hdr.get('T_OUT', np.nan)
                    if 'CHAM_T' in hdr:
                        r['chamt'] = hdr.get('CHAM_T', np.nan)
                    else:
                        r['chamt'] = hdr.get('T_CHAM', np.nan)
            t.write(f, overwrite=True)

fixing /Volumes/LaCie 8TB/wfsdat/20190101/reanalyze_results.csv...
fixing /Volumes/LaCie 8TB/wfsdat/20190102/reanalyze_results.csv...
fixing /Volumes/LaCie 8TB/wfsdat/20190103/reanalyze_results.csv...
fixing /Volumes/LaCie 8TB/wfsdat/20190104/reanalyze_results.csv...
fixing /Volumes/LaCie 8TB/wfsdat/20190105/reanalyze_results.csv...
fixing /Volumes/LaCie 8TB/wfsdat/20190106/reanalyze_results.csv...
fixing /Volumes/LaCie 8TB/wfsdat/20190107/reanalyze_results.csv...
fixing /Volumes/LaCie 8TB/wfsdat/20190108/reanalyze_results.csv...
fixing /Volumes/LaCie 8TB/wfsdat/20190109/reanalyze_results.csv...
fixing /Volumes/LaCie 8TB/wfsdat/20190110/reanalyze_results.csv...
fixing /Volumes/LaCie 8TB/wfsdat/20190111/reanalyze_results.csv...
fixing /Volumes/LaCie 8TB/wfsdat/20190112/reanalyze_results.csv...
fixing /Volumes/LaCie 8TB/wfsdat/20190113/reanalyze_results.csv...
fixing /Volumes/LaCie 8TB/wfsdat/20190114/reanalyze_results.csv...
fixing /Volumes/LaCie 8TB/wfsdat/20190115/reanalyze_results.cs

fixing /Volumes/LaCie 8TB/wfsdat/20190504/reanalyze_results.csv...
fixing /Volumes/LaCie 8TB/wfsdat/20190505/reanalyze_results.csv...
fixing /Volumes/LaCie 8TB/wfsdat/20190506/reanalyze_results.csv...


In [45]:
print("1\
2\
3")

123
